In [0]:
# service principal for integrating with ADLS and access it's data

spark.conf.set("fs.azure.account.auth.type.hpadlsacc.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.hpadlsacc.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "app-key"))
spark.conf.set("fs.azure.account.oauth2.client.secret.hpadlsacc.dfs.core.windows.net", dbutils.secrets.get("hc-secret-scope", "service-cred"))
tenant_id = dbutils.secrets.get("hc-secret-scope", "dir-id")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.hpadlsacc.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

# create department dimensional table

In [0]:
%sql
-- creating dim_department table with external location
CREATE TABLE IF NOT EXISTS gold.dim_department (
  Dept_Id STRING,
  SRC_Dept_Id STRING,
  Name STRING,
  datasource STRING
)
USING DELTA
LOCATION "abfss://gold@hpadlsacc.dfs.core.windows.net/dim_department";


In [0]:
%sql
-- resetting the table for each run
truncate table gold.dim_department;

In [0]:
%sql
-- loading the record fetching from silver layer
insert into gold.dim_department
select distinct Dept_Id ,
SRC_Dept_Id ,
Name ,
datasource 
from silver.departments
where is_quarantined=false

In [0]:
%sql
-- pulling some data
select * from gold.dim_department;